<h2 style="text-align: center;"><b>Word embeddings</b></h2>

In [1]:
!pip install --upgrade nltk gensim bokeh

     |████████████████████████████████| 1.5MB 783kB/s eta 0:00:01
     |████████████████████████████████| 24.7MB 299kB/s eta 0:00:01     |███████████████████             | 14.6MB 1.2MB/s eta 0:00:09     |████████████████████████████▊   | 22.1MB 1.3MB/s eta 0:00:02
     |████████████████████████████████| 17.8MB 2.5MB/s eta 0:00:01     |████████████████████▎           | 11.3MB 2.4MB/s eta 0:00:03
     |████████████████████████████████| 71kB 147kB/s eta 0:00:011
     |████████████████████████████████| 133kB 802kB/s eta 0:00:01
     |████████████████████████████████| 5.7MB 1.5MB/s eta 0:00:01
     |████████████████████████████████| 71kB 215kB/s eta 0:00:011
  Stored in directory: /Users/daniilkorbut/Library/Caches/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
  Stored in directory: /Users/daniilkorbut/Library/Caches/pip/wheels/a7/df/f1/80f108cd241803a5200576ac0eed112f0acea5582575cf08b6
  Stored in directory: /Users/daniilkorbut/Library/Caches/pip/wheels/5f/ea/fb/5b

**NLTK** (Natural Language Toolkit) -- a library with many features to work with natural language (e.g. lemmatization, tokenization, etc.)

https://www.nltk.org

We wull use it for tokenization for now.

In [2]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

quora = list(open("./quora.txt"))
# look at 10th question from the dataset
print(tokenizer.tokenize(quora[10]))

['Which', 'brand', 'should', 'go', 'with', 'the', 'GTX', '960', 'graphic', 'card', ',', 'MSI', ',', 'Zotac', 'or', 'ASUS', '?']


Let's tokenize all the texts from quora.txt file and get array of arrays of tokens:

In [3]:
quora_tokenized = [tokenizer.tokenize(line.lower()) for line in quora]

There are many different models to train to get word embeddings (Word2Vec, GloVe, FastText, etc). For now let's try to train word2vec.

Library **gensim** will help us with that -- it provides models for Word2Vec training and further vectors' investigation

In [4]:
from gensim.models import Word2Vec
# define a model (just like torch model) with parameters
# and train it on our data
model = Word2Vec(quora_tokenized, # data for model to train on
                 size=32,         # embedding vector size
                 min_count=5,     # consider words that occured at least 5 times
                 window=3).wv     # define context as a 3-word window around the target word

Now we have trained model and can play with it:

In [5]:
# we can get a vector for any word in our vocabulary
model.get_vector('tv')

array([-0.27617228,  6.1051373 , -1.1585325 ,  0.19717441, -0.77615315,
        1.3808045 , -2.2622037 , -0.83657634,  1.8669027 ,  1.8883147 ,
       -0.22466594,  0.23334873,  1.20261   , -0.02836741,  0.19491689,
        1.9990473 ,  2.4067266 , -1.899442  ,  1.3972796 , -2.7939408 ,
        0.58265024,  3.9040642 , -2.2348464 , -0.8013072 , -1.0593402 ,
        0.9963031 ,  2.1993587 ,  3.2382784 ,  0.21081461, -1.3073905 ,
        1.3929702 ,  2.6873975 ], dtype=float32)

In [6]:
# and get words that have most similar vectors to vector of the given word
model.most_similar('tv')

[('television', 0.8958842158317566),
 ('netflix', 0.782157301902771),
 ('game', 0.77797532081604),
 ('anime', 0.7679802179336548),
 ('games', 0.7626651525497437),
 ('ps4', 0.738214910030365),
 ('dvd', 0.7342666387557983),
 ('player', 0.7303683161735535),
 ('band', 0.7252438068389893),
 ('music', 0.7231017351150513)]

Nice!

But actually for big tasks as text classifications or something like we would like to have better vectors. If we started training them by ourselves, we would need a LARGE text corpus and LARGE amount of memory and time. But as we are poor students by now, and need to do our homework till the next week, we don't have it all by now.

That is why there are pre-trained models or even pre-computed word vectors collections!

In [7]:
import gensim.downloader as api
model = api.load('glove-twitter-100') # list of available models: https://github.com/RaRe-Technologies/gensim-data#models

[==================================================] 100.0% 387.1/387.1MB downloaded


In [8]:
# who is a person who has money and related to coding, but is really stupid?
# (it's how a model trained on text corpus thinks, not me)
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155143737793),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.49220192432403564),
 ('shopper', 0.4920561909675598),
 ('part-time', 0.49128279089927673),
 ('freelance', 0.4843312203884125),
 ('aupair', 0.4796452522277832)]

Also using gensim we can load pre-trained vectors from vectors file. We can for example load pre-trained google word2vec vectors:

http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/

In [9]:
# from gensim.models import KeyedVectors
# model = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True) 

## Visualization

So let's visualize our embeddings on 2-d space. 

What we need is:
1. get embeddings for a batch of words from the gensim model
2. reduce dimensionality of those embeddings from 100-dim to 2-dim vectors
3. normalize those vectors
4. draw it on a plane!

So let's do that:

In [10]:
# let's get some most popular words from model:
n_words = 1000
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count, # sort by number of word occurencies
               reverse=True)[:n_words]

print(words[::10])

['<user>', 'i', ')', 'no', 'la', 'o', '>', 'are', 'we', '♥', '_', 'now', '~', 'people', "'re", '>>', '[', 'q', 'je', 'never', 'please', 'c', 'hahaha', 'لا', 'then', 'itu', 'had', 'ah', 'esta', 'does', 'apa', 'les', 'work', 'god', 'ha', 'their', 'в', '\\', 'gue', '＾', 'justin', 'thing', 'hope', 'nigga', 'udah', 'same', 'voy', 'find', 'اللي', 'face', 'text', 'kau', 'very', 'may', 'pretty', 'already', 'qué', 'día', 'money', 'name', 'hari', 'wow', 'black', 'vamos', 'nao', 'head', 'trying', 'lot', 'ان', 'mad', 'playing', '✌', 'own', 'friday', 'estar', 'yet', 'une', 'mana', 'لي', 'car', 'once', 'nem', 'summer', 'fan', 'tudo', 'sexy', 'sea', 'excited', 'far', 'lost', 'sei', 'sabe', 'picture', 'dice', 'km', 'fast', 'moi', 'anda', 'noche', 'relationship']


In [11]:
# get embeddings for those words
word_vectors = [model.get_vector(word) for word in words]

For vectors visualization we need to reduce dimensionality of vectors to 2 (or 3). We'll use **PCA** for that

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [12]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(word_vectors)
word_vectors_pca = pca.transform(word_vectors)

Also let's normalize vectors we got from PCA for better visibility

We already did normalization by hand in one of the first homeworks in our course, but for now let's use sklearn api:

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [13]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler().fit(word_vectors_pca)
word_vectors_pca = ss.transform(word_vectors_pca)

Finally we are to draw an (interactive!) space of embeddings, using function below:

In [17]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig


draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

Loading BokehJS ...

Figure(id='1111', ...)

Well, as we can see, this space does not look perfect, sometimes we see (it seems) different words grouped together.

That happend mostly because our way to reduce dimensionality of vectors (PCA)  is not perfect. Let's try to use different algorithm for doing this: TSNE.

TSNE is in some way creates embeddings itself, so we may hope that it will work better)

https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

https://distill.pub/2016/misread-tsne/

In [20]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, verbose=50)
word_vectors_tsne = tsne.fit_transform(word_vectors)

ss = StandardScaler().fit(word_vectors_tsne)
word_vectors_tsne = ss.transform(word_vectors_tsne)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.003s...
[t-SNE] Computed neighbors for 1000 samples in 0.352s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 1.716134
[t-SNE] Computed conditional probabilities in 0.034s
[t-SNE] Iteration 50: error = 67.7193451, gradient norm = 0.3343575 (50 iterations in 9.822s)
[t-SNE] Iteration 100: error = 67.9582138, gradient norm = 0.2989068 (50 iterations in 9.245s)
[t-SNE] Iteration 150: error = 68.5828857, gradient norm = 0.3087775 (50 iterations in 8.546s)
[t-SNE] Iteration 200: error = 69.2910767, gradient norm = 0.3029344 (50 iterations in 9.260s)
[t-SNE] Iteration 250: error = 69.0198822, gradient norm = 0.2894115 (50 iterations in 9.189s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 69.019882
[t-SNE] Iteration 300: error = 1.2050655, gradient norm = 0.0019959 (50 iterations in 7.049s)
[t-SNE] Iteration 350: error = 1.1152738, gradient norm = 0.00048

In [21]:
output_notebook()
draw_vectors(word_vectors_tsne[:, 0], word_vectors_tsne[:, 1], token=words)

Loading BokehJS ...

Figure(id='1228', ...)